<div class="alert alert-block alert-info">
<b>Imports:</b> Common imports for NumPy, MatplotLib, Pandas, and OS along with some others </div>

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import netCDF4
from netCDF4 import Dataset
import numpy.ma as ma
import numpy as np
import pickle
from simpledbf import Dbf5
import shapefile
import util
from util import sort_by_id
import single_fire
from single_fire import *
import datetime
from datetime import date
import math
from math import floor, ceil
import itertools
import numpy as np
import rasterio
from osgeo import osr, ogr, gdal
import os
#from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid

<div class="alert alert-block alert-info">
<b>Imports:</b> Import your .nc dataset for temperature (Look at Notion on where to find this) </div>

In [ ]:
nc_f = "C:/Users/nicoc/Downloads/Complete_TAVG_Daily_LatLong1_2010.nc"  # Your filename
nc_fid = Dataset(nc_f, 'r')  # Dataset is the class behavior to open the file
                    # and create an instance of the ncCDF4 class

<div class="alert alert-block alert-warning">
<b>Keys and dictionaries:</b> Get the keys and dimensions that are tracked

In [ ]:
print(nc_fid.variables.keys())
print("-----")
print(nc_fid.dimensions)

<div class="alert alert-block alert-warning">
<b> Variable Tracking:</b> Set variables to track

In [ ]:
nc_fid.dimensions
long = nc_fid.variables['longitude']
lat = nc_fid.variables['latitude']
tim = nc_fid.variables['day']
day_num = nc_fid.variables['date_number']
temp = nc_fid.variables['temperature']
land_mask = nc_fid.variables['land_mask']
clim = nc_fid.variables['climatology']
day_of_year = nc_fid.variables['day_of_year']

<div class="alert alert-block alert-warning">
<b>Imports:</b> Let's look at the temperature data

In [ ]:
temp

<div class="alert alert-block alert-warning">
<b>Helper functions:</b> Defines some useful helper functions
    

In [ ]:
def world_to_pixel(geo_matrix, x, y):
    """
    Uses a gdal geomatrix (gdal.GetGeoTransform()) to calculate
    the pixel location of a geospatial coordinate
    """
    ul_x= geo_matrix[0]
    ul_y = geo_matrix[3]
    x_dist = geo_matrix[1]
    y_dist = geo_matrix[5]
    pixel = int((x - ul_x) / x_dist)
    line = -int((ul_y - y) / y_dist)
    return pixel, line

def get_elevation(ds, ds2, point, point2, transform, transform2, longitude, latitude):
    '''
    Get's elevation and topography from specific latitude and longitude
    '''
    point.AddPoint(latitude, longitude)
    point.Transform(transform)
    x, y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
    data = np.array(ds.ReadAsArray(x, y, 1, 1))
    try:
        d = data[0][0]
    except Exception as e:
        print("There was an error")
        d = -9999
    point2.AddPoint(latitude, longitude)
    point2.Transform(transform2)
    x2, y2 = world_to_pixel(ds2.GetGeoTransform(), point2.GetX(), point2.GetY())
    data2 = np.array(ds2.ReadAsArray(x, y, 1, 1))
    try:
        d2 = data2[0][0]
    except Exception as e:
        print("Error 2")
        d2 = -9999
    return d, d2




<div class="alert alert-block alert-warning">
<b>Main function:</b> Returns the elevation and topographical matrix based on the tifs for a fire range
    Use get_elevationd and world_to_pixel
 

In [ ]:
def elevation_func(tif_path, xRange, yRange, xDiff, yDiff):
    
    '''
    Returns the elevation and topographical matrix based on the tifs for a fire range
    Use get_elevationd and world_to_pixel
    '''
    
    ds = gdal.Open(r'C:\\Users\\nicoc\\Downloads\\LF2016_Elev_200_CONUS\\Tif\\LC16_Elev_200.tif')
    ds2 = gdal.Open(r'C:\\Users\\nicoc\\Downloads\\LF2019_FBFM13_200_CONUS\\Tif\\LC19_F13_200.tif')
    
    target = osr.SpatialReference(wkt=ds.GetProjection())
    target2 = osr.SpatialReference(wkt = ds2.GetProjection())
    source = osr.SpatialReference()
    source2 = osr.SpatialReference()
    source.ImportFromEPSG(4326)
    source2.ImportFromEPSG(4326)
    transform = osr.CoordinateTransformation(source, target)
    transform2 = osr.CoordinateTransformation(source2, target2)
    #41.3099° N, 122.3106° W
    # 41.409767, -122.201180
    
    point = ogr.Geometry(ogr.wkbPoint)
    point2 = ogr.Geometry(ogr.wkbPoint)
    
    xArray = np.arange(xRange[0], xRange[1] + xDiff, xDiff)
    yArray = np.arange(yRange[0], yRange[1] + yDiff, yDiff)
    xi = 0
    top_matrix = np.zeros((xArray.shape[0], yArray.shape[0]))
    elevation_matrix = np.zeros((xArray.shape[0], yArray.shape[0]))
    for x in xArray:
        yj = 0
        for y in yArray:
            elevation, top = get_elevation(ds, ds2, point, point2, transform, transform2, x, y)
            elevation_matrix[xi][yj] = elevation
            top_matrix[xi][yj] = top         
            yj += 1
        xi += 1
    
    return elevation_matrix, top_matrix


In [ ]:
# ARCHIVED: used to get elevation for given fire
'''
def elevation_func(tif_path, xRange, yRange, xDiff, yDiff):
    
    ds = gdal.Open(r'C:\\Users\\nicoc\\Downloads\\LF2016_Elev_200_CONUS\\Tif\\LC16_Elev_200.tif')
    
    target = osr.SpatialReference(wkt=ds.GetProjection())
    source = osr.SpatialReference()
    source.ImportFromEPSG(4326)
    transform = osr.CoordinateTransformation(source, target)
    #41.3099° N, 122.3106° W
    # 41.409767, -122.201180
    
    point = ogr.Geometry(ogr.wkbPoint)
    xArray = np.arange(xRange[0], xRange[1] + xDiff, xDiff)
    yArray = np.arange(yRange[0], yRange[1] + yDiff, yDiff)
    xi = 0
    elevation_matrix = np.zeros((xArray.shape[0], yArray.shape[0]))
    for x in xArray:
        yj = 0
        for y in yArray:
            elevation_matrix[xi][yj] = get_elevation(ds, point, transform, x, y)
            yj += 1
        xi += 1
    
    return elevation_matrix
'''

In [ ]:
#ARCHIVED: Used to get elevation for particular point
"""
def get_elevation(ds, point, transform, longitude, latitude):
    '''# Extract target reference from the tiff file
    ds = gdal.Open(r'C:\\Users\\nicoc\\Downloads\\LF2016_Elev_200_CONUS\\Tif\\LC16_Elev_200.tif')
    target = osr.SpatialReference(wkt=ds.GetProjection())

    source = osr.SpatialReference()
    source.ImportFromEPSG(4326)

    transform = osr.CoordinateTransformation(source, target)
    #41.3099° N, 122.3106° W
    # 41.409767, -122.201180
    point = ogr.Geometry(ogr.wkbPoint)
    '''
    point.AddPoint(latitude, longitude)
    point.Transform(transform)
    x, y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
    data = np.array(ds.ReadAsArray(x, y, 1, 1))
    try:
        return data[0][0]
    except Exception as e:
        return -9999
"""


<div class="alert alert-block alert-warning">
<b>Main function:</b> Generates a pickle dictionary object with a bunch of info about a fire
 

In [ ]:

def generate_updated_dict(df, shapeRecords, nc_fid, tif_path, save_path, month, year):
    d = {}
    temp = nc_fid.variables['temperature']
    counter = 0
    for eyeD in np.unique(df['Id']):
        d['fireID'] = eyeD
        idx = df[[df['Id'][i] == eyeD and df['Type'][i] == 'FinalArea' for i in range(len(df))]].index  
        final_fire_indices = shapeRecords[idx[0]].shape.points
        sorted_df = sort_by_id(df, eyeD)
        d['Unique Dates'] = sorted_df['IDate'].unique()
        M = np.load('C:\\Users\\nicoc\\Desktop\\Stanford\\OneDrive\\OneDrive - Stanford\\Courses\\CS229\\finalproject\\data\\United_States_Fires\\United_States_{year}_Fires\\{month}\\multi_day\\{file}.npy'.format(file = str(eyeD), month = month, year = year))
        d['multiDay'] = M
        d['IDate'] = shapeRecords[idx[0]].record['IDate']
        d['FDate'] = shapeRecords[idx[0]].record['FDate']
        a = get_fire_shape_and_discretization(final_fire_indices)
        xRange, yRange, xDiff, yDiff = a
        d['xRange'] = xRange
        d['yRange'] = yRange
        d['xDiff'] = xDiff
        d['yDiff'] = yDiff
        d0 = date(2010, 1, 1)
        d1 = d['IDate']
        d2 = d['FDate']
        long = d['xRange'][0] + (d['xRange'][0] + d['xRange'][1])/2
        lat = d['yRange'][0] + (d['yRange'][0] + d['yRange'][1])/2
        temp_array = []
        for i in range((d2 - d1).days):
            long_index = ceil(long)
            lat_index = ceil(lat)
            day = (d1 - d0).days
            temperature = temp[day + i][lat_index][long_index]     
            temp_array.append(temperature)
            
        d['Temperature'] = temp_array
        
        elevation, top = elevation_func(tif_path, xRange, yRange, xDiff, yDiff)
        

        d['Elevation Data'] = elevation
        d['Topographic Data'] = top
        counter += 1
        
                
        print("Fire {day} done".format(day = counter))
        
        # create a binary pickle file 
        f = open("{path}/{eyeD}.pkl".format(path = save_path, eyeD = str(eyeD)),"wb")
        pickle.dump(d,f)
        f.close()



<div class="alert alert-block alert-warning">
<b>Main:</b> MAIN script to iterate through all of nico's files and create the saved dictionaries
 

In [ ]:

nc_f = "C:/Users/nicoc/Downloads/Complete_TAVG_Daily_LatLong1_2010.nc"  # Your filename
tif_path = 'C:\\Users\\nicoc\\Downloads\\LF2016_Elev_200_CONUS\\Tif\\LC16_Elev_200.tif'
nc_fid = Dataset(nc_f, 'r')  # Dataset is the class behavior to open the file
                            # and create an instance of the ncCDF4 class
temp = nc_fid.variables['temperature']
parent_path = 'C:\\Users\\nicoc\\Desktop\\Stanford\\OneDrive\\OneDrive - Stanford\\Courses\\CS229\\finalproject\\data\\United_States_Fires\\United_States_2018_Fires\\'
for subdir, dirs, files in os.walk(parent_path):
    for directory in dirs:
        '''
        if directory not in ['sep']:
            continue
        '''
        
        if directory not in ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']:
            continue
        
        print('Currently on {month}'.format(month = directory))
        for file in os.listdir(r"{path}/{month}/".format(path = subdir, month = directory)): 
            # check the files which are end with specific extension
            if file.endswith(".shp"):
                # print path name of selected files
                shape = shapefile.Reader("{path}/{month}/{file_name}".format(file_name = str(file), path = subdir, month = directory))
                shapeRecords = shape.shapeRecords()
            if file.endswith(".dbf"):
                dbf =  Dbf5("{path}/{month}/{file_name}".format(file_name = str(file), month = directory, path = subdir))
                df = dbf.to_dataframe()

        if not os.path.isdir( "{path}\\{month}\\storage\\".format(month = directory, path = subdir)):
            os.mkdir("{path}\\{month}\\storage\\".format(month = directory, path = subdir))



        generate_updated_dict(df, shapeRecords, nc_fid, tif_path, "{path}\\{month}\\storage\\".format(path = subdir, month = directory), directory, year = '2018')

        print("Saved data for {x0}-{year}!".format(x0 = directory, year = '2018'))




<div class="alert alert-block alert-warning">
<b>Example: </b> Example on how to use the pickle data!
 

In [ ]:
path = "C:\\Users\\nicoc\\Desktop\\Stanford\\OneDrive\\OneDrive - Stanford\\Courses\\CS229\\finalproject\\data\\United_States_Fires\\United_States_2017_Fires\\apr\\storage"
with open("{path}\\{id}.pkl".format(path = path, id =19477533), 'rb') as f:
    data = pickle.load(f)  